In [20]:
%matplotlib qt

In [21]:

#imports
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import trackpy as tp
import pims
import pandas as pd
from pandas import DataFrame, Series  # for convenience


In [22]:
#Load files
frames = pims.open('videos/10x_testing/111sampleA10X_scaled.tif')
diameter_variable = 15

In [23]:
def analyze_tif(video_name):
    frames = pims.open(video_name)
    f = tp.locate(frames[0], diameter_variable, invert=False)
    plt.figure()
    tp.annotate(f, frames[0])
    f_all = tp.batch(frames, diameter=diameter_variable, processes=1)
    t = tp.link(f_all, search_range=6)
    return t

In [24]:
# Locate particles in first frame
%matplotlib qt
f = tp.locate(frames[0], diameter=diameter_variable)
plt.figure()
tp.annotate(f, frames[0])
plt.show()

In [25]:
# Batch process all of the frames to detect particles
f_all = tp.batch(frames, diameter=diameter_variable,processes=1)



Frame 240: 162 features


In [26]:
# Link particles across all the frames
t = tp.link_df(f_all, search_range=5)

tp.plot_traj(t)
plt.show()



Frame 240: 162 trajectories present.


In [27]:
# Opt filter 
t_filtered = tp.filter_stubs(t, threshold=8)

In [28]:
# Plot the trajectories
tp.plot_traj(t_filtered)
plt.show()

In [29]:
d = tp.compute_drift(t_filtered)

In [30]:
d.plot()
plt.show()

In [31]:
tm = tp.subtract_drift(t_filtered.copy(), d)

In [32]:
ax = tp.plot_traj(tm)
plt.show()


In [33]:
tm = tm.rename_axis(index={'particle': 'particle_index'})
im = tp.imsd(tm, 1/1.855, 11)

In [34]:
fig, ax = plt.subplots()
ax.plot(im.index, im, 'k-', alpha=0.1)  # black lines, semitransparent
ax.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]',
       xlabel='lag time $t$')
ax.set_xscale('log')
ax.set_yscale('log')

In [35]:
em = tp.emsd(tm, 1/1.855,11)

In [36]:
fig, ax = plt.subplots()
ax.plot(em.index, em, 'o')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]',
       xlabel='lag time $t$')
ax.set(ylim=(1e-2, 10));

In [37]:
plt.figure()
plt.ylabel(r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]')
plt.xlabel('lag time $t$');
tp.utils.fit_powerlaw(em)  # performs linear best fit in log space, plots]

# D = A/4 [um^2/s]

,n,A
msd,0.793687,1.858302
